In [ ]:
pip install netCDF4 xarray geopy regex pycountry

In [114]:
import requests
import zipfile
import os
import pycountry
import pandas as pd
import regex as re

#The "download the temperature data" function
def DownloadFile(url, writeLocation):
    if requests.get(url).status_code == 200:
        with open(writeLocation, 'wb') as file:
            file.write(requests.get(url).content)
        return True
    else:
        return f"exited with {requests.get(url).status_code} as error"

def ExtractZip(zipFile, extractPath):
    try:
        with zipfile.ZipFile(zipFile, 'r') as zip_ref:
            zip_ref.extractall(extractPath)
        return extractPath
    except Exception as e:
        return f"Error: {e}"

def GenerateTemperatureMapping(stationsFile):
    STAIDCountry = {}
    reg = r"[\s]*(\d+),([\w\W\s]*),([a-zA-Z]{2})"
    EUList = ['BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE']
    with open(stationsFile) as f:
        lines = f.readlines()
        lines = lines[17:]
        f.close()
    for line in lines:
        m = re.match(reg, line)
        if m and m.group(3) in EUList:
            STAIDCountry[m.group(1)] = pycountry.countries.get(alpha_2=m.group(3)).name

    return STAIDCountry

tempLocatie = 'data/temp'
if DownloadFile("https://knmi-ecad-assets-prd.s3.amazonaws.com/download/ECA_blend_tg.zip", tempLocatie+'.zip'):
    ExtractZip(tempLocatie+'.zip', tempLocatie)

GenerateTemperatureMapping(tempLocatie+'/stations.txt')
    





KeyboardInterrupt



In [ ]:
from geopy.geocoders import Nominatim

def get_country_from_coordinates(latitude, longitude):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    country = location.raw.get("address", {}).get("country")

    return country

In [ ]:
import zipfile

def unzip_file(zip_file_path, extract_to_folder):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to_folder)
        print(f"File '{zip_file_path}' has been successfully extracted to '{extract_to_folder}'.")
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
zip_file_path = 'data/temp.zip'  # Replace with the actual path to your ZIP file
extract_to_folder = 'data/temp/'         # Replace with the folder where you want to extract the contents

unzip_file(zip_file_path, extract_to_folder)


In [ ]:
import pycountry
def get_country_name(iso_code):
    try:
        country = pycountry.countries.get(alpha_2=iso_code).name
        if country:
            return country.name
        else:
            return "Country not found"
    except AttributeError:
        return "Invalid ISO code format"

In [ ]:
import os
import pycountry
import pandas as pd
import regex as re
file = 'data/temp/stations.txt'

STAIDCountry = {}
reg = r"[\s]*(\d+),([\w\W\s]*),([a-zA-Z]{2})"
EUList = ['BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE']

with open(file) as f:
    lines = f.readlines()
    lines = lines[17:]
    f.close()

used_list = ()
for line in lines:
    m = re.match(reg, line)
    if m and m.group(3) in EUList:
        STAIDCountry[m.group(1)] = get_country_name(m.group(3))